# [LPR's RLWE](https://eprint.iacr.org/2012/230.pdf) Public Key Cryptosystem

- [On Ideal Lattices and Learning with Errors Over Rings](https://eprint.iacr.org/2012/230.pdf)
- [Learning With Errors and Ring Learning With Errors](https://medium.com/asecuritysite-when-bob-met-alice/learning-with-errors-and-ring-learning-with-errors-23516a502406)
- [A Homomorphic Encryption Illustrated Primer](https://blog.n1analytics.com/homomorphic-encryption-illustrated-primer/)

In [1]:
import numpy as np
from numpy.testing import assert_array_equal

In [2]:
# NOTE: Uncomment to simplfy debugging
# np.random.seed(1)

## Utility Functions

In [3]:
def polymod(poly, poly_mod, coeff_mod):
    """
    Computes the remainder after a polynomial division
    Args:
        poly: Polynomial
        poly_mod: Polynomial modulus
        coeff_mod: Coefficient modulus
    Returns:
        The coefficients of the remainder when `poly` is divided by `poly_mod`
    """
    return np.poly1d(np.floor(np.polydiv(poly, poly_mod)[1]) % coeff_mod)

def test():
    coeff_mod = 10
    # x^16 + 1
    poly_mod = np.poly1d([1] + (15 * [0]) + [1])
    # 2x^14
    a = np.poly1d([2] + (14 * [0]))
    # x^4
    b = np.poly1d([1] + (4 * [0]))
    # 2x^14 * x^4 = 2x^18
    result_mul = np.polymul(a, b)
    assert_array_equal(result_mul, np.poly1d([2] + (18 * [0])))
    # 2x^18 % x^16 + 1 = -2x^2
    result_mod = polymod(result_mul, poly_mod, coeff_mod)
    assert_array_equal(result_mod, np.poly1d([8, 0, 0]))

test()

In [4]:
def addition(poly_mod, coeff_mod):
    """
    Creates a function which performs polynomial addition and auto-applys polynomial- and coefficient modulus
    Args:
        poly_mod: Polynomial modulus
        coeff_mod: Coefficient modulus
    Returns:
        A function which takes polynomials `a` and `b` and adds them together
    """
    return lambda a, b: np.poly1d(polymod(np.polyadd(a, b), poly_mod, coeff_mod))

def test():
    coeff_mod = 8
    # x^4 + 1
    poly_mod = np.poly1d([1] + (3 * [0]) + [1])
    a = np.poly1d([1, 2, 3, 4])
    b = np.poly1d([1, 2, 3, 4])
    add = addition(poly_mod, coeff_mod)
    result = add(a, b)
    assert_array_equal(result, np.poly1d([2, 4, 6, 0]))

test()

In [5]:
def multiplication(poly_mod, coeff_mod):
    """
    Creates a function which performs polynomial multiplication and auto-applys polynomial- and coefficient modulus
    Args:
        poly_mod: Polynomial modulus
        coeff_mod: Coefficient modulus
    Returns:
        A function which takes polynomials `a` and `b` and multiplies them
    """
    return lambda a, b: np.poly1d(polymod(np.polymul(a, b), poly_mod, coeff_mod))

def test():
    coeff_mod = 8
    # x^4 + 1
    poly_mod = np.poly1d([1] + (3 * [0]) + [1])
    a = np.poly1d([1, 2, 3, 4])
    b = np.poly1d([1, 2, 3, 4])
    mul = multiplication(poly_mod, coeff_mod)
    result = mul(a, b)
    assert_array_equal(result, np.poly1d([4, 0, 4, 6]))

test()

## Security Parameters

In [6]:
n = 4
t = 7
# Highest coefficient power used
d = 2 ** n
# Coefficient modulus
c_q = 874
delta = c_q // t
# Polynomial modulus
p_q = np.poly1d([1] + ([0] * (d - 1)) + [1])

print(f'n: {n}')
print(f't: {t}')
print(f'd: {d}')
print(f'delta: {delta}')
print(f'c_q: {c_q}')
print(f'p_q: \n{p_q}')

n: 4
t: 7
d: 16
delta: 124
c_q: 874
p_q: 
   16
1 x  + 1


In [7]:
assert c_q == delta * t + (c_q % t)
assert p_q.order == d

In [8]:
# Creating our polynomial addition and multiplication functions via our security parameters
add = addition(p_q, c_q)
mul = multiplication(p_q, c_q)

## Secret Key

In [9]:
sk = np.poly1d(np.random.randint(0, 2, d))

print(sk)

   15     14     13     12     10     8     6     5     2
1 x  + 1 x  + 1 x  + 1 x  + 1 x  + 1 x + 1 x + 1 x + 1 x


## Public Key

In [10]:
a = np.poly1d(np.random.randint(0, c_q, d) % c_q)

print(a)

    15       14       13       12       11       10       9      8
29 x  + 195 x  + 692 x  + 407 x  + 193 x  + 237 x  + 316 x + 76 x
        7       6       5      4       3       2
 + 589 x + 292 x + 640 x + 81 x + 201 x + 494 x + 440 x + 248


In [11]:
e = np.poly1d(np.random.normal(0, 2, d).astype(int) % c_q)

print(e)

   14       13       12     10     8       7     6       5       4
1 x  + 873 x  + 871 x  + 1 x  + 1 x + 873 x + 2 x + 872 x + 873 x
        3
 + 869 x + 2 x + 3


In [12]:
pk_0 = add(-mul(a, sk), e)

print(pk_0)

     15       14       13       12       11       10       9       8
513 x  + 298 x  + 458 x  + 720 x  + 308 x  + 823 x  + 247 x + 114 x
        7       6       5       4       3       2
 + 842 x + 179 x + 862 x + 222 x + 805 x + 630 x + 330 x + 464


In [13]:
pk_1 = a

print(pk_1)

    15       14       13       12       11       10       9      8
29 x  + 195 x  + 692 x  + 407 x  + 193 x  + 237 x  + 316 x + 76 x
        7       6       5      4       3       2
 + 589 x + 292 x + 640 x + 81 x + 201 x + 494 x + 440 x + 248


In [14]:
pk = (pk_0, pk_1)

print('pk_0:\n')
print(pk[0])
print()
print('pk_1:\n')
print(pk[1])

pk_0:

     15       14       13       12       11       10       9       8
513 x  + 298 x  + 458 x  + 720 x  + 308 x  + 823 x  + 247 x + 114 x
        7       6       5       4       3       2
 + 842 x + 179 x + 862 x + 222 x + 805 x + 630 x + 330 x + 464

pk_1:

    15       14       13       12       11       10       9      8
29 x  + 195 x  + 692 x  + 407 x  + 193 x  + 237 x  + 316 x + 76 x
        7       6       5      4       3       2
 + 589 x + 292 x + 640 x + 81 x + 201 x + 494 x + 440 x + 248


In [15]:
# We should be able to extract the error `e` from the public key via the secret key
# NOTE: Doing so will make it possible to identify the noise when decrypting later on
def test():
    extr_e = add(mul(pk[1], sk), pk[0])    
    assert_array_equal(extr_e, e)

test()

## Encryption

In [16]:
# 2x^2 + 5
m = np.poly1d((np.array([0] * (d - 3) + [2] + [0] + [5])) % t)

print(m)

   2
2 x + 5


In [17]:
u = np.poly1d(np.random.randint(0, 2, d))

print(u)

   15     14     11     9     8     5     4     3
1 x  + 1 x  + 1 x  + 1 x + 1 x + 1 x + 1 x + 1 x + 1


In [18]:
e_1 = np.poly1d(np.random.normal(0, 2, d).astype(int) % c_q)

print(e_1)

   15     12     11     10       9       7       5
1 x  + 2 x  + 1 x  + 1 x  + 873 x + 873 x + 868 x + 871 x + 3


In [19]:
e_2 = np.poly1d(np.random.normal(0, 2, d).astype(int) % c_q)

print(e_2)

   15       14       12     10       9     8       7     5
4 x  + 873 x  + 873 x  + 1 x  + 869 x + 2 x + 873 x + 1 x + 3 x + 872


In [20]:
c_0 = add(add(mul(pk[0], u), e_1), mul(delta, m))

print(c_0)

    15       14       13       12       11       10       8       7
32 x  + 851 x  + 797 x  + 778 x  + 417 x  + 378 x  + 426 x + 606 x
        6       5       4       3       2
 + 798 x + 132 x + 809 x + 751 x + 166 x + 372 x + 319


In [21]:
c_1 = add(mul(pk[1], u), e_2)

print(c_1)

     15       14       13       12       11       10       9       8
772 x  + 544 x  + 564 x  + 348 x  + 120 x  + 316 x  + 513 x + 848 x
        7       6       5       4       3       2
 + 341 x + 556 x + 480 x + 640 x + 746 x + 776 x + 392 x + 211


## Decryption

In [22]:
m_prime = np.poly1d(np.round(add(mul(c_1, sk), c_0) * t / c_q) % t)

print(m_prime)

assert_array_equal(m_prime, m)

   2
2 x + 5
